<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/timaslj/timaslj_lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Timas Ljungdahl**

# **Abstract**

In this report, 4 algorithms for solving equations iteratively were implemented and tested. The algorithms were Jocobi iteration, Gauss-Seidel iteration, Newton's method for scalar functions, and Newton's method for vector functions. All algorithms were tested with random data and generally generated desired results with around 8 decimal precision.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [0]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import unittest
import random
import math

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this report, systems of linear equations on the form $Ax = b$, non-linear scalar functions on the form $ \mathbb{R}^{n} \rightarrow \mathbb{R}$ and non-linear vector functions on the form $ \mathbb{R}^{n} \rightarrow \mathbb{R}^{n} $ are investigated. 

In the case of very large or sparse system of linear equations $Ax = b$, it makes sense to use iterative methods instead of direct methods, as we want to avoid computing $A^{-1}$ as it can be computationally expensive. Instead we can use iterative methods to generate a sequence of approximations where the error will decrease every iteration. 

One type of such iterative methods is Richardson iteration that we formulate as $X^{k+1} = (I-\alpha A)x^{k} + \alpha b$ where $M = I - \alpha A$ is called the iteration matrix. The method will converge if $||M|| < 1$. In this lab, two iterative methods based on the same theory are implemented; Jacobi and Gauss-Seidel method. The iteration continues until the residual ||$Ax_{approx} - b|| < TOL$ where $TOL$ is a small number close to $0$.  

The Jacobi method is based on matrix splitting $A = D + R$ where $D$ is chosen to be a diagonal matrix which is easy to invert. The approximate solution to $Ax=b$ can then iteratively be obtained with $X^{k+1} = (I-D^{-1} A)x^{k} + D^{-1}b$. In this case $M = I-D^{-1}A$. 

The Gauss-Seidel method is also based on matrix splitting $A = L + U$ where $L$ is a lower triangular matrix with none-zero diagonal elements. The approximate solution to $Ax=b$ can then iteratively be obtained with $X^{k+1} = (I-L^{-1} A)x^{k} + L^{-1}b$. In this case $M = I-L^{-1}A$.  

In order to iteratively solve for a root of a non-linear scalar function, Newton's method was implemented. It is based on computing the functions value and derivate in a point $x$ where the iteration becomes $x_{n+1} = x_n - f(x_n)/f^\prime(x_n)$ where $x_{n+1}$ is a better approximation than $x_n$. In the case of a multivariate function, the Jacobian is computed. If $f(x)$ is a vector function the iteration becomes $x_{n+1} = x_n - f^\prime(x_n)^{-1}f(x_n)$ where $f^\prime(x_n)^{-1}$ is the inverse Jacobian matrix. This can be rewritten as $f^\prime(x_n)^{-1}(x_{n+1}-x_n) = f(x_n)$ which resembles a system of linear equation of the form $Ax = b$, where $\Delta x_{n+1} = x_{n+1}- x_n$ can be solved for.    

# **Methods**

All methods were implemented according to theory. 

For the Newton's methods, the approximate Jacobian is computed by computing the function value difference when inputing a slightly different $x$. 

The matrix iteration methods were tested by asserting the residual $||Ax_{approx} - b||$ and $||x_{approx}-y||$ to be 0, where $y$ is the exact solution, for 1000 random vectors. In order to ensure that the system converges, the system was preconditioned. 

For the Newtons method for scalar functions, random functions were generated and the residual $|f(x)|$ were asserted to 0. $|x-y|$ were very hard to test as the exact solution for the random functions were hard to obtain. 

# **Results**

In [62]:
##can be done without saving R,D and D_inverse -> much more space efficient
def jacobi_iteration(A,b, TOL = 1e-8):
  n = A.shape[0]  
  D_inverse = np.zeros((n,n))
  for i in range(n):
    D_inverse[i,i] = 1/A[i,i]

  M = np.identity(n) - np.matmul(D_inverse, A)

  c = np.matmul(D_inverse,b)

  x = np.random.rand(n)

  while np.linalg.norm(np.matmul(A,x)-b) > TOL:
    x = np.matmul(M, x) + c

  return x

def gauss_seidel_iteration(A, b, TOL = 1e-8):
  n = A.shape[0]  
  L_inverse = A.copy()
  for i in range(n):
    for j in range(n):
      if (j > i):
        L_inverse[i,j] = 0

  L_inverse = np.linalg.inv(L_inverse)

  M = np.identity(n) - np.matmul(L_inverse, A)

  c = np.matmul(L_inverse, b)

  x = np.random.rand(n)

  while np.linalg.norm(np.matmul(A,x)-b) > TOL:
    x = np.matmul(M, x) + c

  return x
    
class Test(unittest.TestCase):

  def test_random_vectors(self):
    for n in range(1000):
      size = random.randint(2, 50) 
      A = np.random.rand(size, size)
      #left preconditioning so system converges
      alpha = 1.1
      B = np.linalg.inv(A)*alpha #approx inverse of A so ||I-alpha*B*A|| < 1
      C = np.matmul(A,B)
      x = np.random.rand(size)
      b = np.matmul(C,x)

      x_gauss_approx = gauss_seidel_iteration(C,b)
      x_jacobi_approx = jacobi_iteration(C,b)

      x_exact = np.linalg.solve(C,b)
    
      #||x-y||
      self.assertAlmostEqual(np.linalg.norm(x_gauss_approx-x_exact), 0, 9)
      self.assertAlmostEqual(np.linalg.norm(x_jacobi_approx-x_exact), 0, 9)

      b_gauss_approx = np.matmul(C,x_gauss_approx)
      b_jacobi_approx = np.matmul(C, x_jacobi_approx)

      #||Ax-b||
      self.assertAlmostEqual(np.linalg.norm(b_gauss_approx-b), 0, 9)
      self.assertAlmostEqual(np.linalg.norm(b_jacobi_approx-b), 0, 9)

    #print(x)
    #x_prim = jacobi_iteration(A,b)
    
    
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.858s

OK


In [66]:
def scalar_jacobian(f, x, dx=1e-8):
  fx = f(x)
  J = np.zeros(x.shape[0])
  dxi = x.copy()

  for i in range(len(x)):
    dxi[i] = dxi[i] + dx
    J[i] = abs(f(dxi)-fx)/dx 
    dxi[i] = x[i]
  
  return J

def scalar_newtons_method(f, x0, TOL = 1e-8):
  x = x0

  while abs(f(x)) > TOL:
    df = scalar_jacobian(f, x)
    for i in range(x.shape[0]): 
      if not(math.isclose(df[i],0)): #divide by zero
        x[i] -= f(x)/df[i]
      else:
        return None 
      
  return x

class Test(unittest.TestCase):
  def get_random_func(n):
    exponents = []
    scalar = random.randint(0,1000)
    for i in range(n):
      exp = random.randint(1,5)
      exponents.append(exp)

    def random_func(x):
      sum = 0
      for i in range(n):
        sum += x[i]**exponents[i]
      return sum + scalar
    return random_func

  def test_random_functions(self):
    for n in range(100000):
      size = random.randint(2,10)
      f = get_random_func(size)
      x0 = np.zeros(size, dtype='float64')

      root_approx = scalar_newtons_method(f,x0)

      if root_approx is None:
        continue

      f_root = f(root_approx)
    
      if f_root == f_root: #not NaN
        #|f(x)|
        self.assertAlmostEqual(abs(f_root), 0, 8)
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 6.981s

OK


In [76]:
def vector_jacobian(f, x, dx=1e-6):
  fx = f(x)
  number_of_variables = fx[1]
  J = np.zeros((fx[0].shape[0], number_of_variables), dtype='float64')
  dxi = x.copy()

  for i in range(x.shape[0]):
    dxi[i] = dxi[i] + dx
    fdxi = f(dxi)
    J[:,i] = (fdxi[0]-fx[0])/dx 
    dxi[i] = x[i]
  
  return J

def vector_newtons_method(f, x0, TOL = 1e-8):
  x = x0

  fx = f(x)

  #nums_of_its = 0 

  while np.linalg.norm(fx[0]) > TOL:
    fx = f(x)
    df = vector_jacobian(f,x)
    dx = np.linalg.solve(df, -fx[0])
    x = x + dx
    #nums_of_its += 1

  #print("Nums of its: ", nums_of_its)
  return x

class Test(unittest.TestCase):

  def function1(x):
    return (np.array([(x[0]**2)*x[1], 5*x[0]+math.sin(x[1])], dtype='float64'),2)

  def function2(x):
    return (np.array([x[0]*math.cos(x[1]), x[0]*math.sin(x[1])], dtype='float64'),2)

  def function3(x):
    return (np.array([5*x[1], 4*(x[0]**2)*2*math.sin(x[1]*x[2]), x[1]*x[2]], dtype='float64'),3)

  #left preconditioning so system converges
  def test_functions(self):
    for n in range(1000):
      x0_2_variables = np.random.rand(2)
      x0_3_variables = np.random.rand(3)

      root_f1_approx = vector_newtons_method(function1, x0_2_variables)
      root_f2_approx = vector_newtons_method(function2, x0_2_variables)
      root_f3_approx = vector_newtons_method(function3, x0_3_variables)

      #||f(x)||
      self.assertAlmostEqual(np.linalg.norm(function1(root_f1_approx)[0]), 0, 8)
      self.assertAlmostEqual(np.linalg.norm(function2(root_f2_approx)[0]), 0, 8)
      self.assertAlmostEqual(np.linalg.norm(function3(root_f3_approx)[0]), 0, 8)

        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 1.186s

OK


# **Discussion**

All algorithms were implemented and tested with random data several times. The precision of the algorithms differs but generally the output had a precision of around 8 decimals.    